In [3]:
import sys 
import os
import subprocess
import numpy as np 
import time
from itertools import product 

option = 2
output_dir = '../output/run_clixo/option{}'.format(option); 
if not os.path.exists(output_dir): os.makedirs(output_dir)
stringdb_dir = '../output/extracting_string_interactions_subnetworks/'

In [4]:
# Obtaining the clixo program 
clixo_script = '/gpfs/data01/glasslab/home/joreyna/projects/BNFO286/clixo_0.3/clixo'

# Listing the string interactions file
interactions_fn = os.path.join(stringdb_dir, 'synapse_inters_option_{}.tsv')
interactions_fn = interactions_fn.format(option)
interactions_fn 

'../output/extracting_string_interactions_subnetworks/synapse_inters_option_2.tsv'

## Code to run a single parameter set 


In [38]:
alpha = 0.01
beta = 0.2

# Naming the clixo output (can change if you need)
clixo_fn = os.path.join(output_dir, 
    'string_synapse_interactions_combined_score.clixo_alpha{}_beta{}.txt'.format(alpha, beta))
cmd = '{} {} {} {} > {}'.format(clixo_script, interactions_fn, alpha, beta, clixo_fn)
print('Running: {}'.format(cmd))

if not os.path.exists(clixo_fn): 
    sp = subprocess.Popen(cmd, stderr=subprocess.PIPE, stdout=subprocess.PIPE, shell=True)
    msgs = sp.communicate()
    print('\n'.join([x.decode() for x in msgs]))

Running: /gpfs/data01/glasslab/home/joreyna/projects/BNFO286/clixo_0.3/clixo ../output/extracting_string_interactions_subnetworks/synapse_inters_option_2.tsv 0.01 0.2 > ../output/run_clixo/option2/string_synapse_interactions_combined_score.clixo_alpha0.01_beta0.2.txt


### Testing different values of alpha and beta

In [17]:
def get_alpha_beta(l):
    """
    Yield function to get the correct alpha and beta names.
    
    """
    for x in l: 
        alpha = '{:.2f}'.format(x[0])
        beta =  '{:.2f}'.format(x[1])
        yield((alpha, beta))
        
    yield(None, None)

In [ ]:
# import shutil

In [54]:
# Gathering all the 
parameters = product(np.arange(0.01, 0.05, 0.01), np.arange(0.3, 0.5, 0.01))
parameters = get_alpha_beta(parameters)

# for alpha, beta in parameters:
    
#     # Running CliXo jobs with a new set of parameters
#     clixo_fn = 'string_synapse_interactions_combined_score'
#     clixo_fn += '.clixo_alpha{}_beta{}.txt'
#     clixo_fn = clixo_fn.format(alpha, beta)
#     clixo_fn = os.path.join(output_dir + '/archive/', clixo_fn)
    
#     if os.path.exists(clixo_fn):
#         shutil.move(clixo_fn, output_dir)

In [55]:
jobs = {}
concurrent_jobs = 20
num_loops = 0 
while True: 
    
    # Get the status of jobs  
    jobs = {job: job.poll() for job in jobs}
            
    # Remove jobs
    job_keys = jobs.keys()
    for job_key in list(job_keys):  
        status = jobs[job_key]  
        # Job is no longer running, remove it 
        if status != None: 
            jobs.pop(job_key)      
            
    # Start new jobs 
    num_new_jobs = concurrent_jobs - len(jobs)  
    
    #print('num_new_jobs', num_new_jobs)
    
    for i in range(num_new_jobs):
            
        # Pulling the alpha and beta values 
        alpha, beta = next(parameters)
        #print('alpha, beta', alpha, beta)
            
        # Checking whether there are more parameters to run  
        if alpha == None: 
            break 
            
        # Running CliXo jobs with a new set of parameters
        clixo_fn = 'string_synapse_interactions_combined_score'
        clixo_fn += '.clixo_alpha{}_beta{}.txt'
        clixo_fn = clixo_fn.format(alpha, beta)
        clixo_fn = os.path.join(output_dir, clixo_fn)

        if not os.path.exists(clixo_fn):
            cmd = '{} {} {} {} > {}'.format(clixo_script, 
                                            interactions_fn, 
                                            alpha, 
                                            beta, 
                                            clixo_fn)
            print('Running: {}'.format(cmd))
            job = subprocess.Popen(cmd, 
                                   stderr=subprocess.PIPE, 
                                   stdout=subprocess.PIPE, 
                                   shell=True)

            # Add the status of the job as running (=None)
            jobs[job] = None 
            
    if len(jobs) == concurrent_jobs: 
        time.sleep(180)
    else:
        time.sleep(5)
        
    print('Iteraction number: {}, number of jobs running: {}'.\
          format(num_loops, len(jobs)))
    num_loops+= 1

Running: /gpfs/data01/glasslab/home/joreyna/projects/BNFO286/clixo_0.3/clixo ../output/extracting_string_interactions_subnetworks/synapse_inters_option_2.tsv 0.01 0.41 > ../output/run_clixo/option2/string_synapse_interactions_combined_score.clixo_alpha0.01_beta0.41.txt
Running: /gpfs/data01/glasslab/home/joreyna/projects/BNFO286/clixo_0.3/clixo ../output/extracting_string_interactions_subnetworks/synapse_inters_option_2.tsv 0.01 0.42 > ../output/run_clixo/option2/string_synapse_interactions_combined_score.clixo_alpha0.01_beta0.42.txt
Running: /gpfs/data01/glasslab/home/joreyna/projects/BNFO286/clixo_0.3/clixo ../output/extracting_string_interactions_subnetworks/synapse_inters_option_2.tsv 0.01 0.43 > ../output/run_clixo/option2/string_synapse_interactions_combined_score.clixo_alpha0.01_beta0.43.txt
Running: /gpfs/data01/glasslab/home/joreyna/projects/BNFO286/clixo_0.3/clixo ../output/extracting_string_interactions_subnetworks/synapse_inters_option_2.tsv 0.01 0.44 > ../output/run_clixo/

Iteraction number: 7, number of jobs running: 20
Running: /gpfs/data01/glasslab/home/joreyna/projects/BNFO286/clixo_0.3/clixo ../output/extracting_string_interactions_subnetworks/synapse_inters_option_2.tsv 0.04 0.48 > ../output/run_clixo/option2/string_synapse_interactions_combined_score.clixo_alpha0.04_beta0.48.txt
Running: /gpfs/data01/glasslab/home/joreyna/projects/BNFO286/clixo_0.3/clixo ../output/extracting_string_interactions_subnetworks/synapse_inters_option_2.tsv 0.04 0.49 > ../output/run_clixo/option2/string_synapse_interactions_combined_score.clixo_alpha0.04_beta0.49.txt
Iteraction number: 8, number of jobs running: 2


StopIteration: 